# Setup

In [ ]:
# ! pip install pandas==0.25

In [ ]:
import warnings                                  
warnings.filterwarnings('ignore')

import numpy as np #for numerical computations like log,exp,sqrt etc

# data in/out & eda
import pandas as pd  #for reading & storing data, pre-processing
import pandas_profiling

# visualisations
import matplotlib.pyplot as plt # for plots
import plotly.express as px
import plotly.graph_objects as go

import statsmodels.api as sm    # statistics and econometrics
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA

import tensorflow as tf
import sklearn.preprocessing
from sklearn.metrics import r2_score

from keras.layers import Dense,Dropout,SimpleRNN,LSTM
from keras.models import Sequential

# Importing everything from forecasting quality metrics
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error

from tqdm import tqdm_notebook

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Load Dataset

In [ ]:
# let's only read one of the region's power consumptions, to keep this notebook concise
df = pd.read_csv('/content/drive/MyDrive/715_project/PRSA_Data_Wanliu_20130301-20170228.csv', header=0, index_col=0, squeeze=True)

# select columns
df = df[["year", "month", "day", "hour", "PM2.5"]]

# into datetime
df['Datetime']=pd.to_datetime(df[['year','month','day', 'hour']])

# keep only 2 columns
df = df[["Datetime", "PM2.5"]]

# sort by date & time
df['Datetime'] = pd.to_datetime(df['Datetime'])
df.sort_values(by=['Datetime'], axis=0, ascending=True, inplace=True)
df.reset_index(inplace=True, drop=True)

# display the first couple of rows
df.head()

,Datetime,PM2.5
0,2013-03-01 00:00:00,8.0
1,2013-03-01 01:00:00,9.0
2,2013-03-01 02:00:00,3.0
3,2013-03-01 03:00:00,11.0
4,2013-03-01 04:00:00,3.0


In [ ]:
# how many observations in df
df.count()

Datetime    35064
PM2.5       34682
dtype: int64

## Deduplicating

In [ ]:
if len(set(df)) == len(df):
    print("success")
else:
    print("duplicate found")

duplicate found


In [ ]:
# deduplicate, only keeping the last measurement per datetime
df.drop_duplicates(subset='Datetime', keep='last', inplace=True)

## Find and Fill Missing DateTime Instances

In [ ]:
# let's see if we have a continuous dataset
df = df.set_index('Datetime')
print(f'df.index.freq is set to: {df.index.freq}')

df.index.freq is set to: None


The fact our datetime index's frequency is set to None is an indication there are some missing data points somewhere (otherwise Python could deduce it).

In [ ]:
df.isnull()

# Count total NaN in a DataFrame
print(" \nCount total NaN in a DataFrame : \n\n",
       df.isnull().sum().sum())

 
Count total NaN in a DataFrame : 

 382


In [ ]:
# uninterruped custom date range
date_range = pd.date_range(start=min(df.index), 
                           end=max(df.index), 
                           freq='H')

In [ ]:
# print(f'The difference in length between the custom date range and our dataset is {(len(date_range)-len(df))}:')
# print(date_range.difference(df.index))

There are 382 missing values. Let's reindex our dataset and then perform imputation.


In [ ]:
# this will append the previously missing datetimes, and create null values in our target variable
df = df.reindex(date_range)

# we fill in the blanks with values that lie on a linear curve between existing data points
df['PM2.5'].interpolate(method='linear', inplace=True)

# now we have a neatly continuous datetime index
print(f'The df.index.freq is now: {df.index.freq}, indicating that we no longer have missing instances')

The df.index.freq is now: <Hour>, indicating that we no longer have missing instances


In [ ]:
df.isnull()

# Count total NaN in a DataFrame
print(" \nCount total NaN in a DataFrame : \n\n",
       df.isnull().sum().sum())

 
Count total NaN in a DataFrame : 

 0


In [ ]:
df.count()

PM2.5    35064
dtype: int64

## Extracting Time Features

> Indented block



We can split up the date-timestamp column into its different components. This will allow us to find patterns for different groups.

In [ ]:
# see: https://www.kaggle.com/robikscube/starter-hourly-energy-consumption
df['dow'] = df.index.dayofweek
df['doy'] = df.index.dayofyear
df['year'] = df.index.year
df['month'] = df.index.month
df['quarter'] = df.index.quarter
df['hour'] = df.index.hour
df['weekday'] = df.index.day_name()
df['woy'] = df.index.weekofyear
df['dom'] = df.index.day # Day of Month
df['date'] = df.index.date 

# let's add the season number
df['season'] = df['month'].apply(lambda month_number: (month_number%12 + 3)//3)

# Exploratory data analysis

A quick one-liner with pandas_profiling to get an overview of our dataset

In [ ]:
# pandas_profiling.ProfileReport(df)

# Quick Visuals

## Plotting the energy consumption over time

In [ ]:

# plotly doesn't allow us to access the index, so let's copy it into a column 
df['date_and_time'] = df.index

# plotting
fig = px.line(df,
              x='date_and_time',
              y='PM2.5',
              title=f'Levels of PM2.5 over time [{min(df.year)} - {max(df.year)}]')
fig.update_traces(line=dict(width=1))
fig.update_layout(xaxis_title='Date & Time (yyyy/mm/dd hh:MM)',
                  yaxis_title='PM2.5 (ug/m3)')
fig.show()

We can definitely identify a seasonal pattern here. Counter-intuitively, though, no immediately apparent trend.

## Date & Time Patterns

We can use our previously extracted date and time features to see if recurring patterns emerge from aggregated data. Take, for instance, the power demand throughout the day for each weekday:

In [ ]:
# aggregated data
_ = df\
    .groupby(['hour', 'weekday'], as_index=False)\
    .agg({'PM2.5':'max'})

# plotting
fig = px.line(_, 
              x='hour', 
              y='PM2.5', 
              color='weekday', 
              title='Max Hourly PM2.5 per Weekday')
fig.update_layout(xaxis_title='Hour',
                  yaxis_title='PM2.5')
fig.show()

In [ ]:
# aggregated data
_ = df\
    .groupby(['hour', 'weekday'], as_index=False)\
    .agg({'PM2.5':'median'})

# plotting
fig = px.line(_, 
              x='hour', 
              y='PM2.5', 
              color='weekday', 
              title='Median Hourly PM2.5 per Weekday')
fig.update_layout(xaxis_title='Hour',
                  yaxis_title='PM2.5')
fig.show()

In [ ]:
# aggregated data
_ = df\
    .groupby(['hour', 'weekday'], as_index=False)\
    .agg({'PM2.5':'min'})

# plotting
fig = px.line(_, 
              x='hour', 
              y='PM2.5', 
              color='weekday', 
              title='Min Hourly PM2.5 per Weekday')
fig.update_layout(xaxis_title='Hour',
                  yaxis_title='PM2.5')
fig.show()

We can quickly tell demand for electricity is lower during the weekends, and dips a little sooner on friday afternoons.


*   Season 1 = Winter
*   Season 2 = Spring
*   Season 3 = Summer
*   Season 4 = Fall





In [ ]:
# aggregated data
_ = df\
    .groupby(['hour', 'season'], as_index=False)\
    .agg({'PM2.5':'max'})

# plotting
fig = px.line(_,
              x='hour', 
              y='PM2.5', 
              color='season', 
              title='Max PM2.5 per Season')
fig.update_layout(xaxis_title='Hour',
                  yaxis_title='PM2.5')
fig.show()

In [ ]:
# aggregated data
_ = df\
    .groupby(['hour', 'season'], as_index=False)\
    .agg({'PM2.5':'median'})

# plotting
fig = px.line(_,
              x='hour', 
              y='PM2.5', 
              color='season', 
              title='Median PM2.5 per Season')
fig.update_layout(xaxis_title='Hour',
                  yaxis_title='PM2.5')
fig.show()

In [ ]:
# aggregated data
_ = df\
    .groupby(['hour', 'season'], as_index=False)\
    .agg({'PM2.5':'min'})

# plotting
fig = px.line(_,
              x='hour', 
              y='PM2.5', 
              color='season', 
              title='Min PM2.5 per Season')
fig.update_layout(xaxis_title='Hour',
                  yaxis_title='PM2.5')
fig.show()

Looks like PM2.5 is highest during the winters

# Decomposing the Time-Series
## Trends, seasonality, noise

Decomposing the Time-Series
Data points over time can be interesting in the sense that their patterns are complemented by a trend (upward or downward) and/or seasonality. As we have established in our EDA, these aspects seem to play a role in this dataset.

Because the seasonal variation in our dataset looks constant over time, we will use the additive model for decomposition (as opposed to the multiplicative model, which is useful for cases where seasonal variation increases over time).



These are the components of a time series

Trend - Consistent upwards or downwards slope of a time series
Seasonality - Clear periodic pattern of a time series(like sine funtion)
Noise - Outliers or missing values

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# seasonal_decompose needs a dataframe with a datetime index
series = df[['PM2.5']]
frequency = 24*365

# decomposing the time-series, with the frequency being 24 hours per 365 days
decomposed = seasonal_decompose(series, model='additive', freq=frequency)

In [ ]:
# plotting the different elements constituting our time-series
def plot_decompositions(decompositions, titles, line_widths):
    for d, t, lw in zip(decompositions, titles, line_widths):
        
        # draw a line plot of the data
        fig = px.line(d,
              y='PM2.5',
              title=t,
              height=300)
        
        # adjust line width
        fig.update_traces(line=dict(width=lw))
        
        # change layout of axes and the figure's margins 
        # to emulate tight_layout
        fig.update_layout(
            xaxis=dict(
                showticklabels=False,
                linewidth=1
            ),
            yaxis=dict(title=''),
            margin=go.layout.Margin(
                l=40, r=40, b=0, t=40, pad=0
            ),
        )
        
        # display
        fig.show()

# calling the function 
plot_decompositions(decompositions=[decomposed.observed,
                                    decomposed.trend, 
                                    decomposed.seasonal, 
                                    decomposed.resid],
                    titles=['Observed',
                            'Trend', 
                            'Seasonality',
                            'Residuals'],
                    line_widths=[1, 2, 1, 1])

- There is clearly an downward trend in the above plot.
- You can also see the uniform seasonal change.
- Non-uniform noise that represent outliers and missing values

# Random Walk
A random walk is a mathematical object, known as a stochastic or random process, that describes a path that consists of a succession of random steps on some mathematical space such as the integers.

Pt = Pt-1 + εt

Random walks can't be forecasted because well, noise is random.



Dickey-Fuller Test:
*   $H_0$: β = 0 (This is a random walk)
*   $H_1$: β < 0 (This is not a random walk)



Augmented Dickey-Fuller test:

An augmented Dickey–Fuller test (ADF) tests the null hypothesis that a unit root is present in a time series sample. It is basically Dickey-Fuller test with more lagged changes on RHS.

In [ ]:
#Perform Augmented Dickey–Fuller test:
print('Results of Dickey Fuller Test:')
dftest = adfuller(df['PM2.5'], autolag='AIC')

dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
    
print(dfoutput)

Results of Dickey Fuller Test:
Test Statistic                   -18.926629
p-value                            0.000000
#Lags Used                        51.000000
Number of Observations Used    35012.000000
Critical Value (1%)               -3.430537
Critical Value (5%)               -2.861623
Critical Value (10%)              -2.566814
dtype: float64


For a Time series to be stationary, its ADCF test should have:

1. p-value to be low (according to the null hypothesis)
2. The critical values at 1%,5%,10% confidence intervals should be as close as possible to the Test Statistics

From the above ADCF test result, we see that p-value is very low. However critical values are no where close to the Test Statistics. Hence, we can safely say that our Time Series at the moment is not stationary

# Test for Stationarity

A formal way of testing stationarity of a dataset is using plotting the moving average or moving variance and see if the series mean and variance varies with time. This approach will be handled by the TestStationaryPlot() method. The second way to test stationarity is to use the statistical test (the Dickey-Fuller Test). The null hypothesis for the test is that the time series is non-stationary. The test results compare a Test Statistic and Critical Values (cutoff value) at different confidence levels. If the ‘Test Statistic’ is less than the ‘Critical Value’, we can reject the null hypothesis and say that the series is stationary. This technique will be handled by the TestStationaryAdfuller( ) method given below.

In [ ]:
def TestStationaryPlot(ts):
    rol_mean = ts.rolling(window = 12, center = False).mean()
    rol_std = ts.rolling(window = 12, center = False).std()
    
    plt.plot(ts, color = 'blue',label = 'Original Data')
    plt.plot(rol_mean, color = 'red', label = 'Rolling Mean')
    plt.plot(rol_std, color ='black', label = 'Rolling Std')
    plt.xticks(fontsize = 10)
    plt.yticks(fontsize = 10)
    
    plt.xlabel('Time in Years', fontsize = 10)
    plt.ylabel('Total Emissions', fontsize = 10)
    plt.legend(loc='best', fontsize = 10)
    plt.title('Rolling Mean & Standard Deviation', fontsize = 10)
    plt.show(block= True)

In [ ]:
def TestStationaryAdfuller(ts, cutoff = 0.01):
    ts_test = adfuller(ts, autolag = 'AIC')
    ts_test_output = pd.Series(ts_test[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    
    for key,value in ts_test[4].items():
        ts_test_output['Critical Value (%s)'%key] = value
    print(ts_test_output)
    
    if ts_test[1] <= cutoff:
        print("Strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root, hence it is stationary")
    else:
        print("Weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

In [ ]:
TestStationaryPlot(df['PM2.5'])

ADF test is used to determine the presence of unit root in the series, and hence helps in understand if the series is stationary or not. The null and alternate hypothesis of this test are:

Null Hypothesis: The series has a unit root.

Alternate Hypothesis: The series has no unit root.

If the null hypothesis in failed to be rejected, this test may provide evidence that the series is non-stationary.

In [ ]:
TestStationaryAdfuller(df["PM2.5"])

Test Statistic                   -18.926629
p-value                            0.000000
#Lags Used                        51.000000
Number of Observations Used    35012.000000
Critical Value (1%)               -3.430537
Critical Value (5%)               -2.861623
Critical Value (10%)              -2.566814
dtype: float64
Strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root, hence it is stationary


Our p-value is definitely less than 0.5 and is even less than 0.01 so we can say with pretty good confidence that we can reject the null (unit root, non-stationary data) and can assume our data is stationary. Additionally, our ADF is much less than our 1% confidence value of -3.43, so we have another confirmation that we can reject the null.

Now that we know its stationary, we need to see if its correlated (remember there’s an assumption of dependance / correlation for autoregression). Let’s look at a lagplot.

In [ ]:
import pandas as pd
pd.plotting.lag_plot(df['PM2.5'])

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f01c8f30d40> (for post_execute):


ValueError: ignored

# Kwiatkowski-Phillips-Schmidt-Shin (“KPSS”)

KPSS is another test for checking the stationarity of a time series. The null and alternate hypothesis for the KPSS test are opposite that of the ADF test.

Null Hypothesis: The process is trend stationary.

Alternate Hypothesis: The series has a unit root (series is not stationary).

In [ ]:
# KPSS test
from statsmodels.tsa.stattools import kpss
def kpss_test(series, **kw):    
    statistic, p_value, n_lags, critical_values = kpss(series, **kw)
    # Format Output
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
    print(f'Result: The series is {"not " if p_value < 0.05 else ""}stationary')

kpss_test(df["PM2.5"])

KPSS Statistic: 0.9721619126615032
p-value: 0.01
num lags: 52
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/stattools.py:1709: InterpolationWarning:

p-value is smaller than the indicated p-value



## Plot the ACF and PACF charts and find the optimal parameters
- Autocorrelation Function (ACF): It is a measure of the correlation between the the time series (ts) with a lagged version of itself. For instance at lag 4, ACF would compare series at time instant ‘t1’…’t2’ with series at instant ‘t1-4’…’t2-4’ (t1-4 and t2 being end points of the range).
- Partial Autocorrelation Function (PACF): This measures the correlation between the ts with a lagged version of itself but after eliminating the variations already explained by the intervening comparisons. Eg at lag 4, it will check the correlation but remove the effects already explained by lags 1 to 3.

Therefore, the next step will be determing the tuning parameters (p and q) of the model by looking at the autocorrelation and partial autocorrelation graphs. The chart below provides a brief guide on how to read the autocorrelation and partial autocorrelation graphs inorder to select the parameters.

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df["PM2.5"].iloc[13:], lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df["PM2.5"].iloc[13:], lags=40, ax=ax2)

Autocorrelation - The autocorrelation function (ACF) measures how a series is correlated with itself at different lags.

Partial Autocorrelation - The partial autocorrelation function can be interpreted as a regression of the series against its past lags. The terms can be interpreted the same way as a standard linear regression, that is the contribution of a change in that particular lag while holding others constant.

The next step in ourtime series analysis is to review Autocorrelation Function (ACF) and Partial Autocorrelation Function (PACF) plots.

ACF is a measure of the correlation between the timeseries with a lagged version of itself. For instance at lag 5, ACF would compare series at time instant ‘t1’…’tn’ with series at instant ‘t1-5’…’tn-5’ (t1-5 and tn being end points).

PACF, on the other hand, measures the correlation between the timeseries with a lagged version of itself but after eliminating the variations explained by the intervening comparisons. Eg. at lag 5, it will check the correlation but remove the effects already explained by lags 1 to 4.

# Eliminating trend and seasonality: Differencing

One of the most common method of dealing with both trend and seasonality is differencing. In this technique, we take the difference of the original observation at a particular instant with that at the previous instant. This mostly works well to improve stationarity. First order differencing can be done as follows:

In [ ]:
df_first_difference = df["PM2.5"] - df["PM2.5"].shift(1)  
TestStationaryPlot(df_first_difference.dropna(inplace=False))

ValueError: ignored

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7fbbceeded40> (for post_execute):


ValueError: ignored

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df_first_difference.iloc[13:], lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df_first_difference.iloc[13:], lags=40, ax=ax2)

In [ ]:
TestStationaryAdfuller(df_first_difference.dropna(inplace=False))

#  Find optimal parameters and build SARIMA model



When looking to fit time series dataset with seasonal ARIMA model, our first goal is to find the values of SARIMA(p,d,q)(P,D,Q)s that optimize our metric of interest. Before moving directly how to find the optimal values of the parameters let us see the two situations in stationarities: A strictly stationary series with no dependence among the values. This is the easy case wherein we can model the residuals as white noise. The second case being a series with significant dependence among values and needs statistical models like ARIMA to forecast future oucomes.

Auto-Regressive Integrated Moving Average (ARIMA): The ARIMA forecasting for a stationary time series is a linear funcion similar to linear regression. The predictors mainly depend on the parameters (p,d,q) of the ARIMA model:

- Number of Auto-Regressive (AR) terms (p): AR terms are just lags of dependent variable. For instance if p is 4, the predictors for x(t) will depend on x(t-1)….x(t-4). This term allows us to incorporate the effect of past values into our model. This would be similar to stating that the weather is likely to be warm tomorrow if it has been warm the past 4 days.
- Number of Moving Average(MA) terms (q): MA terms are lagged forecast errors in prediction function. This term allows us to set the error of our model as a linear combination of the error values observed at previous time points in the past. For instance if q is 4, the predictors for x(t) will be e(t-1)….e(t-4) where e(i) is the difference between the moving average at ith instant and actual value.
- Number of Differences (d): These are the number of nonseasonal differences, i.e., if we took the first order difference. So either we can pass the first order difference variable and put d=0 or pass the original observed variable and put d=1. Both will generate same results. This term explains the number of past time points to subtract from the current value. This would be similar to stating that it is likely to be same temperature tomorrow if the difference in temperature in the last three days has been very small.

# Train, Validation, Test split

In [ ]:
#train_test_split

# train values: 24865
# valid values: 8784
# test values: 1415

train_start,tr_end = '2013-03-01','2015-12-31'
valid_start,val_end = '2016-01-01','2016-12-31'
test_start,te_end = '2017-01-01','2017-02-28'

train_set = df['PM2.5'][train_start:tr_end].dropna()
valid_set = df['PM2.5'][valid_start:val_end].dropna()
test_set = df['PM2.5'][test_start:te_end].dropna()

In [ ]:
# check right dates
valid_set.head()
valid_set.tail()

2016-12-31 19:00:00    409.0
2016-12-31 20:00:00    416.0
2016-12-31 21:00:00    434.0
2016-12-31 22:00:00    468.0
2016-12-31 23:00:00    475.0
Freq: H, Name: PM2.5, dtype: float64

## Grid search
To find the optimal parameters for ARIMA models using the graphical method is not trivial and it is time consuming. We will select the optimal parameter values systematically using the grid search (hyperparameter optimization) method. The grid search iteratively explore different combinations of the parameters. For each combination of parameters, we will fit a new seasonal ARIMA model with the SARIMAX() function from the statsmodels module and assess its overall quality. Once we have explored the entire landscape of parameters, our optimal set of parameters will be the one that yields the best performance for our criteria of interest. Let's begin by generating the various combination of parameters that we wish to assess:

In [ ]:
# this worked
import itertools
from itertools import product

# set parameter range
p = range(0,3)
q = range(0,3)
d = range(0,3)
s = 12
# list of all parameter combos
pdq = list(itertools.product(p, d, q))
seasonal_pdq = list(itertools.product(p, d, q, s))
# SARIMA model pipeline
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(train_set,
                                    order=param,
                                     seasonal_order=param_seasonal)
            results = mod.fit(max_iter = 50, method = 'powell')
            print('SARIMA{},{} - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

TypeError: ignored

When evaluating and comparing statistical models fitted with different parameters, each can be ranked against one another based on how well it fits the data or its ability to accurately predict future data points. We will use the AIC (Akaike Information Criterion) value, which is conveniently returned with ARIMA models fitted using statsmodels. The AIC measures how well a model fits the data while taking into account the overall complexity of the model. A model that fits the data very well while using lots of features will be assigned a larger AIC score than a model that uses fewer features to achieve the same goodness-of-fit. The lowest AIC refore, we are interested in finding the model that yields the lowest AIC value.

The order argument specifies the (p, d, q) parameters, while the seasonal_order argument specifies the (P, D, Q, S) seasonal component of the Seasonal ARIMA model. After fitting each SARIMAX()model, the code prints out its respective AIC score.

         Current function value: 4.410601
         Iterations: 4
         Function evaluations: 331
SARIMA(2, 1, 1),(2, 1, 1, 7) - AIC:219344.3720483094

# SARIMA parameters

- p is most probably 4 since it is the last significant lag on the PACF, after which, most others are not significant.
- d equals 1 because we had first differences
- q should be somewhere around 4 as well as seen on the ACF
- P might be 2, since 24-th and 48-th lags are somewhat significant on the PACF
- D again equals 1 because we performed seasonal differentiation
- Q is probably 1. The 24-th lag on ACF is significant while the 48-th is not

In [ ]:
from itertools import product

# setting initial values and some bounds for them
ps = range(2, 5)
d=1 
qs = range(2, 5)
Ps = range(0, 2)
D=1 
Qs = range(0, 2)
s = 24 # season length is still 24

# creating list with all the possible combinations of parameters
parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

36

In [ ]:
def optimizeSARIMA(y, parameters_list, d, D, s):
    """Return dataframe with parameters and corresponding AIC
        
        y - time series
        parameters_list - list with (p, q, P, Q) tuples
        d - integration order in ARIMA model
        D - seasonal integration order 
        s - length of season
    """
    
    results = []
    best_aic = float("inf")

    for param in tqdm_notebook(parameters_list):
        # we need try-except because on some combinations model fails to converge
        try:
            model=sm.tsa.statespace.SARIMAX(y, order=(param[0], d, param[1]), 
                                            seasonal_order=(param[2], D, param[3], s)).fit(disp=-1)
        except:
            continue
        aic = model.aic
        # saving best model, AIC and parameters
        if aic < best_aic:
            best_model = model
            best_aic = aic
            best_param = param
        results.append([param, model.aic])

    result_table = pd.DataFrame(results)
    result_table.columns = ['parameters', 'aic']
    # sorting in ascending order, the lower AIC is - the better
    result_table = result_table.sort_values(by='aic', ascending=True).reset_index(drop=True)
    
    return result_table

In [ ]:
%%time
warnings.filterwarnings("ignore") 
result_table = optimizeSARIMA(train_set.sample(frac=.01), parameters_list, d, D, s)

ValueError: ignored

In [ ]:
def best_sarima_model(train_data,p,q,P,Q,d=1,D=1,s=12):
    best_model_aic = np.Inf 
    best_model_bic = np.Inf 
    best_model_hqic = np.Inf
    best_model_order = (0,0,0)
    models = []
    for p_ in p:
        for q_ in q:
            for P_ in P:
                for Q_ in Q:
                    try:
                        no_of_lower_metrics = 0
                        model = SARIMAX(endog=train_data,order=(p_,d,q_), seasonal_order=(P_,D,Q_,s),
                                        enforce_invertibility=False).fit()
                        models.append(model)
                        if model.aic <= best_model_aic: no_of_lower_metrics+=1
                        if model.bic <= best_model_bic: no_of_lower_metrics+=1
                        if model.hqic <= best_model_hqic:no_of_lower_metrics+=1
                        if no_of_lower_metrics >= 2:
                            best_model_aic = np.round(model.aic,0)
                            best_model_bic = np.round(model.bic,0)
                            best_model_hqic = np.round(model.hqic,0)
                            best_model_order = (p_,d,q_,P_,D,Q_,s)
                            current_best_model = model
                            models.append(model)
                            print("Best model so far: SARIMA" +  str(best_model_order) + 
                                  " AIC:{} BIC:{} HQIC:{}".format(best_model_aic,best_model_bic,best_model_hqic)+
                                  " resid:{}".format(np.round(np.exp(current_best_model.resid).mean(),3)))

                    except:
                        pass

    print('\n')
    print(current_best_model.summary())                
    return current_best_model, models

In [ ]:
best_model, models = best_sarima_model(train_data=train_set,p=range(3),q=range(3),P=range(3),Q=range(3))

UnboundLocalError: ignored